In [32]:
import networkx as nx
import matplotlib.pyplot as plt
from pysmt.shortcuts import *
from pysmt.typing import INT
import textwrap

In [33]:
# Creating the directed graph
G = nx.DiGraph()

# Symbolic variables
r = Symbol("r", INT)
r_prime = Symbol("r'", INT)
s = Symbol("s", INT)
s_prime = Symbol("s'", INT)
t = Symbol("t", INT)
t_prime = Symbol("t'", INT)
q = Symbol("q", INT)

# Nodes: each represents a state or phase of the algorithm
nodes = [
    ("system", "havoc r, r', s, s', t, t'"),  # Descriptive annotation
    ("do", "skip"),  # Descriptive annotation for the main execution state
    ("error", "r == 0 ou overflow"),  # Descriptive annotation for error state
    ("stop", "r' == 0 (fim do algoritmo)"),  # Descriptive annotation for stop state
    ("update", And(  # Symbolic expression for transitions
        Equals(r, r_prime), 
        Equals(r_prime, r - q * r_prime),
        Equals(s, s_prime),
        Equals(s_prime, s - q * s_prime),
        Equals(t, t_prime),
        Equals(t_prime, t - q * t_prime)
    ))
]

# Adding nodes and symbolic annotations
for label, annotation in nodes:
    G.add_node(label, annotation=annotation)

# Transitions between nodes with conditions
edges = [
    ("system", "do", GE(r, Int(0))),  # Verify r >= 0
    ("do", "stop", Equals(r_prime, Int(0))),  # When r' == 0, algorithm terminates
    ("do", "error", Equals(r, Int(0))),  # If r == 0, it's an error
    ("do", "update", And(NotEquals(r_prime, Int(0)), GE(r, Int(0)))),  # While r' != 0, continue algorithm
    ("update", "do", "")  # Transition back to 'do' after update
]

# Adding edges and their conditions
for source, target, condition in edges:
    G.add_edge(source, target, condition=condition)

# Visualization setup
pos = nx.spring_layout(G, k=0.8)
node_sizes = {node: 10000 for node in G.nodes}  # Default node size
node_sizes["update"] = 22000  # Increase size for 'update' node
plt.figure(figsize=(10, 8))
nx.draw(G, pos=pos, with_labels=True, node_size=[node_sizes[node] for node in G.nodes], connectionstyle="arc3,rad=0.2")

# Edge condition labels
edge_conds = nx.get_edge_attributes(G, name="condition")
nx.draw_networkx_edge_labels(G, pos, edge_conds)

# Node annotations
annotations = nx.get_node_attributes(G, "annotation")
for node, annotation in annotations.items():
    xp, yp = pos[node]
    # Convert annotations to strings if they're symbolic expressions
    wrapped_annotation = textwrap.fill(str(annotation), width=80)
    plt.text(xp, yp - 0.1, wrapped_annotation, fontsize=8, ha="center", va="bottom")

# Display the graph
plt.show()


KeyboardInterrupt: 

In [ ]:
def graph_safety(G, n):
    print(f"> A testar {n} iterações.")
    with Solver(name="z3") as solver:
        # Define symbolic variables
        r = Symbol("r", INT)
        r_prime = Symbol("r_prime", INT)
        s = Symbol("s", INT)
        s_prime = Symbol("s_prime", INT)
        t = Symbol("t", INT)
        t_prime = Symbol("t_prime", INT)
        q = Symbol("q", INT)

        do = FALSE()

        # Extracting the guard condition from the graph
        guarda = FALSE()
        for source, target, data in G.edges(data=True):
            if source == "system" and target == "do":
                guarda = data["condition"]

        # Iterating through the iterations and checking system safety
        for i in range(n):
            system = And(guarda, do)

            # Check if the system is unsafe in this iteration
            if solver.solve([system]):
                print(f"> Iteração {i}: sistema inseguro.")
                return

            clauses = []
            # Iterate over the edges of the graph to verify transitions
            for source, target, data in G.edges(data=True):
                if source == "do":
                    condition = data["condition"]
                    if target == "error":
                        clauses.append(Implies(condition, TRUE()))
                    elif target == "stop":
                        clauses.append(Implies(condition, FALSE()))
                    else:
                        # Handle the "update" node directly here
                        if target == "update":
                            # Substitution logic for the update node
                            subs = {
                                r: r_prime,
                                r_prime: r - q * r_prime,
                                s: s_prime,
                                s_prime: s - q * s_prime,
                                t: t_prime,
                                t_prime: t - q * t_prime,
                            }
                            new_do = substitute(do, subs)
                            clauses.append(Implies(condition, new_do))
                        else:
                            clauses.append(Implies(condition, do))

            # Compute the new state
            new_do = And(clauses)

            # Check if the state has stabilized
            R = And(do, Not(new_do))  # Previous state != new state
            L = And(new_do, Not(do))  # New state != previous state

            if solver.is_unsat(Or(R, L)):
                print(f"> Iteração {i}: sistema seguro.")
                return

            # Update the 'do' condition for the next iteration
            do = Or(do, new_do)

        print("> Não foram encontrados estados inseguros.")

graph_safety(G, 10)


> A testar 10 iterações.
> Iteração 2: sistema inseguro.


In [ ]:
# Definir variáveis simbólicas
r = Symbol('r', INT)
r_prime = Symbol('r_prime', INT)
s = Symbol('s', INT)
s_prime = Symbol('s_prime', INT)
t = Symbol('t', INT)
t_prime = Symbol('t_prime', INT)
q = Symbol('q', INT)

# Condições iniciais (valores iniciais de r, r_prime, s, s_prime, t, t_prime)
a, b = Symbol('a', INT), Symbol('b', INT)
initial_conditions = And(
    Equals(r, a),
    Equals(r_prime, b),
    Equals(s, Int(1)),
    Equals(s_prime, Int(0)),
    Equals(t, Int(0)),
    Equals(t_prime, Int(1))
)

# Equações de transição
transition_equations = And(
    Equals(r_prime, r - q * r_prime),  # Atualização de r'
    Equals(s_prime, s - q * s_prime),  # Atualização de s'
    Equals(t_prime, t - q * t_prime),  # Atualização de t'
    Equals(q, Div(r,r_prime))            # Cálculo do quociente (divisão inteira)
)

# Condições de segurança (evitar r = 0 e garantir que as variáveis são positivas)
overflow_condition = And(
    r > 0, r_prime > 0, s > 0, s_prime > 0, t > 0, t_prime > 0
)

# Condições de segurança: garantir que r não seja zero
safety_conditions = And(
    Not(Equals(r, Int(0))),
    Not(Equals(r_prime, Int(0))),
    overflow_condition
)

# Criar o solver Z3 usando o PySMT
solver = Solver(name="z3")

# Adicionar todas as condições (inicialização, transições e segurança)
solver.add_assertion(initial_conditions)
solver.add_assertion(transition_equations)
solver.add_assertion(safety_conditions)

# Verificar se o sistema é seguro (se satisfaz todas as condições)
if solver.check_sat():
    print("O programa é seguro.")
else:
    print("O programa pode atingir um estado de erro.")

O programa pode atingir um estado de erro.
